In [10]:
import sys
import os

# Add project root to path
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.config.models import default_llm, default_embedding_model
from src.agents.agent import get_agent
from src.agents.tools import db_tool_manager
from src.config.models import get_llm

print("✅ Imports successful")


# Option 1: Use Ollama
default_llm["provider"] = "ollama"
default_llm["model"] = "llama3.1:8b"

# Option 2: Use OpenAI (uncomment to use)
# default_llm["provider"] = "openai"
# default_llm["model"] = "gpt-4o-mini"


print(f"Using: {default_llm['provider']}:{default_llm['model']}")
print(f"Embedding: {default_embedding_model['provider']}:{default_embedding_model['model']}")

✅ Imports successful
Using: ollama:llama3.1:8b
Embedding: openai:text-embedding-3-small


In [11]:
# Get the agent
agent = get_agent(default_llm["provider"], default_llm["model"])
print("✅ Agent initialized successfully")

✅ Agent initialized successfully


In [ ]:
from src.agents.agent import stream_agent
from src.ui.utils import extract_content_from_chunk, extract_todos

question = "List the top 5 customers by order count"
config = {"configurable": {"thread_id": "test-stream-1"}}

print(f"Question: {question}\n")
print("="*80)

full_response = ""
current_plan = ""
chunk_count = 0

try:
    for chunk in stream_agent(agent, question, config):
        chunk_count += 1
        
        # Extract plan updates
        new_plan = extract_todos(chunk)
        if new_plan and new_plan != current_plan:
            current_plan = new_plan
            print(f"\n📋 Plan Update (chunk {chunk_count}):")
            print(current_plan)
            print("-" * 40)
        
        # Extract content
        new_contents = extract_content_from_chunk(chunk)
        for content in new_contents:
            full_response += content
            print(content, end="", flush=True)
    
    print("\n" + "="*80)
    print(f"\n✅ Processed {chunk_count} chunks")
    
    if current_plan:
        print("\n📋 Final Plan:")
        print(current_plan)
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

In [12]:
from src.agents.agent import stream_agent

question = "List the top 5 customers by order count"
config = {"configurable": {"thread_id": "test-stream-fixed"}}

print(f"Question: {question}\n")
print("="*80 + "\n")

for chunk in stream_agent(agent, question, config):
    # Chunk structure: (stream_mode, (message, metadata)) or (stream_mode, dict)
    if isinstance(chunk, tuple) and len(chunk) == 2:
        stream_mode, data = chunk
        
        # Handle token streaming: (AIMessageChunk, metadata)
        if isinstance(data, tuple) and len(data) >= 1:
            message = data[0]
            if hasattr(message, 'content') and message.content:
                print(message.content, end="", flush=True)
        
        # Handle update streaming: dict with messages
        elif isinstance(data, dict):
            for node_name, node_data in data.items():
                if isinstance(node_data, dict) and "messages" in node_data:
                    for msg in node_data["messages"]:
                        if hasattr(msg, 'content') and msg.content:
                            # This is the full message (like the Action: line)
                            if not msg.content.startswith('Action:'):  # Skip tool calls
                                print(msg.content, end="", flush=True)

print("\n" + "="*80)

Question: List the top 5 customers by order count




INFO: HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Searching schema vector store for question: top 5 customers by order count


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


--- Relevant Table 1: Sales.SalesOrderHeader ---
Table: Sales.SalesOrderHeader

    Schema Details:
    Table: Sales.SalesOrderHeader

Columns:
  - SalesOrderID: INTEGER NOT NULL
  - RevisionNumber: TINYINT NOT NULL
  - OrderDate: DATETIME NOT NULL
  - DueDate: DATETIME NOT NULL
  - ShipDate: DATETIME NULL
  - Status: TINYINT NOT NULL
  - OnlineOrderFlag: NULL NOT NULL
  - SalesOrderNumber: NVARCHAR(25) COLLATE "SQL_Latin1_General_CP1_CI_AS" NOT NULL
  - PurchaseOrderNumber: NULL NULL
  - AccountNumber: NULL NULL
  - CustomerID: INTEGER NOT NULL
  - SalesPersonID: INTEGER NULL
  - TerritoryID: INTEGER NULL
  - BillToAddressID: INTEGER NOT NULL
  - ShipToAddressID: INTEGER NOT NULL
  - ShipMethodID: INTEGER NOT NULL
  - CreditCardID: INTEGER NULL
  - CreditCardApprovalCode: VARCHAR(15) COLLATE "SQL_Latin1_General_CP1_CI_AS" NULL
  - CurrencyRateID: INTEGER NULL
  - SubTotal: MONEY NOT NULL
  - TaxAmt: MONEY NOT NULL
  - Freight: MONEY NOT NULL
  - TotalDue: MONEY NOT NULL
  - Comment: N

INFO: HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Action: sql_db_query  
Action Input: SELECT TOP 5 c.CustomerID, COUNT(soh.SalesOrderID) as OrderCount FROM Sales.Customer c JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID GROUP BY c.CustomerID ORDER BY OrderCount DESC
